In [ ]:
import json
import os

from shapely.geometry import shape
import geopandas as gpd

import irv_autopkg_client
from irv_autopkg_client.models import Job
from irv_autopkg_client.api.jobs import submit_processing_job_v1_jobs_post as submit_job
from irv_autopkg_client.api.jobs import get_status_v1_jobs_job_id_get as get_job_status
from irv_autopkg_client.api.packages import get_package_v1_packages_boundary_name_get as get_package_by_name

from utils import download_data

In [ ]:
data_dir = "data"
country_iso = "pak"

In [ ]:
# create and persist a client to make requests with

base_url = "https://global.infrastructureresilience.org/extract"
client = irv_autopkg_client.Client(base_url)

In [ ]:
# submitting a job using one of those data processors

job = Job(
    country_iso,
    [
        "wri_aqueduct.version_2",
        "jrc_ghsl_population.r2022_epoch2020_1km",
        "gri_osm.roads_and_rail_version_1"
    ]
)
# json_body is a poorly named kwarg, its type is Job, not dict
response = submit_job.sync_detailed(client=client, json_body=job)
submitted_job_id = json.loads(response.content)["job_id"]

In [ ]:
# checking the status of that job

# TODO: recreate client API, needs updating for this to work?
response = get_job_status.sync(submitted_job_id, client=client)
response.job_status

In [ ]:
# request country data by name

country_data = get_package_by_name.sync(country_iso, client=client)
[a for a in dir(country_data) if not a.startswith("_")]

In [ ]:
# read the country boundary geometry and save to disk

boundary = shape(country_data.boundary.geometry.additional_properties)
df = gpd.GeoDataFrame({"name": [country_iso], "geometry": [boundary]})

# save it for later use
os.makedirs(os.path.join(data_dir, country_iso), exist_ok=True)
df.to_file(os.path.join(data_dir, country_iso, "territory.gpkg"))

In [ ]:
# parse the datapackage contents for filenames

resources = country_data.datapackage.additional_properties["resources"]
datasets = {
    d["name"]: d["path"] for d in resources
}
for name, paths in datasets.items():
    print(f"dataset: {name}")
    for path in sorted(paths):
        print(f"  file: {path.split('/')[-1]}")

In [ ]:
# download the extracted flood maps
dataset = "wri_aqueduct"
download_data(
    datasets[dataset],
    os.path.join(data_dir, country_iso, dataset),
    r"rcp4p5_wtsub_2050",
)

In [ ]:
# download the network representations
dataset = "gri_osm"
download_data(
    datasets[dataset],
    os.path.join(data_dir, country_iso, dataset)
)